In [23]:
from scipy.sparse.linalg import eigsh
from utility import *
import time
def SCG(dataset, K, rounding_strategy, N=None, A=None):
    """ find K polarized communities """
    if dataset != 'sbm': # real-world dataset
        print('------ Running {}.txt ------'.format(dataset))
        # read graph
        N, A = read_graph("datasets/{}.txt".format(dataset))
    else: # synthetic modified SBM
        pass
    # eigendecompose the core KI-1 matrix
    D,U = EigenDecompose_Core(K)
    U = U[:, D.argsort()]
    D = np.sort(D)
    # initialization
    Y = np.zeros((N,K)) # to be determined, it must satisfy (1) Y_{i,1}=1/sqrt(K) if not neutral; 0 otherwise (2) Y_{i,2:} in {0, U_{1,2:}, ..., U_{K,2:}}
    C = np.array([-1 for i in range(N)]) # cluster assignment, C_i in {-1, 1, ..., K}, where -1 represent neutral
    mask = np.ones((N)) # list of nodes to be assigned
    maskA = A.copy() # adjacency matrix of the remaining graph

    for z in reversed(range(1,K)): # assign from the 1st, ..., the (K-1)-th, the K-th clusters
        lD, lU = eigsh(maskA, k=1, which='LA') # the eigenvector of the largest eigenvalue
        sD, _ = eigsh(maskA, k=1, which='SA') # the eigenvector of the smallest eigenvalue
        lD, sD = lD[0], sD[0]
        v = lU[:,0].reshape((-1))
        zi = K-z # this iteration decides the zi-th cluster
        # Round v to {-1,0,z}^n
        if rounding_strategy=='min_angle':
            v_round = round_by_min_angle(v, z, -1, mask, N)
        elif rounding_strategy=='randomized':
            v_round = round_by_randomized_vector(v, z, -1, mask, maskA, N)
        elif rounding_strategy=='max_obj':
            v_round = round_by_max_obj_one_threshold(v, z, -1, mask, maskA, N)
        elif rounding_strategy=='bansal':
            v_round = round_by_cc_bansal(z, -1, mask, maskA, N)
        # assign to the new cluster(s)
        for i in range(N):
            if v_round[i]==0: continue
            if z>1:
                if v_round[i]>0: C[i], Y[i,:] = zi, U[zi-1,:].copy() # assign to the zi-th cluster
            else:
                if v_round[i]>0: C[i], Y[i,:] = zi, U[zi-1,:].copy() # assign to the (K-1)-th cluster
                else: C[i], Y[i,:] = zi+1, U[zi,:].copy() # assign to the K-th cluster
        # check current objective value
        print('{}-th iteration obj={:.1f}, x^TAx/x^Tx={:.1f} in ({:.1f}, {:.1f})'.format(
            zi, compute_Obj(Y, A, K), compute_RayleighsQuotient(v_round, maskA), sD, lD))
        # set the assigned nodes to be skipped in the next iteration
        for i in range(N):
            if v_round[i]>0:
                # remove all edges incident to the assigned nodes
                maskA[i,:] = maskA[i,:].multiply(0)
                maskA[:,i] = maskA[:,i].multiply(0)
                mask[i] = 0 # remove the assigned node from the remaining list
    return C, Y, A, N, K

In [56]:
from scipy.sparse.linalg import eigsh
from utility import *
import time
def SCG2(dataset, K, rounding_strategy, N=None, A=None):
    """ find K polarized communities """
    if dataset != 'sbm': # real-world dataset
        print('------ Running {}.txt ------'.format(dataset))
        # read graph
        N, A = read_graph("datasets/{}.txt".format(dataset))
    else: # synthetic modified SBM
        pass
    # eigendecompose the core KI-1 matrix
    D,U = EigenDecompose_Core(K)
    U = U[:, D.argsort()]
    D = np.sort(D)
    # initialization
    Y = np.zeros((N,K)) # to be determined, it must satisfy (1) Y_{i,1}=1/sqrt(K) if not neutral; 0 otherwise (2) Y_{i,2:} in {0, U_{1,2:}, ..., U_{K,2:}}
    C = np.array([-1 for i in range(N)]) # cluster assignment, C_i in {-1, 1, ..., K}, where -1 represent neutral
    mask = np.ones((N)) # list of nodes to be assigned
    maskA = A.copy() # adjacency matrix of the remaining graph

    for z in reversed(range(1,K)): # assign from the 1st, ..., the (K-1)-th, the K-th clusters
        lD, lU = eigsh(maskA, k=1, which='LA') # the eigenvector of the largest eigenvalue
        sD, _ = eigsh(maskA, k=1, which='SA') # the eigenvector of the smallest eigenvalue
        lD, sD = lD[0], sD[0]
        v = lU[:,0].reshape((-1))
        zi = K-z # this iteration decides the zi-th cluster
        # Round v to {-1,0,z}^n
        print("HERE: ", v[np.where(v > 0)])
        if rounding_strategy=='min_angle':
            v_round = round_by_min_angle(v, z, -1, mask, N)
        elif rounding_strategy=='randomized':
            v_round = round_by_randomized_vector(v, z, -1, mask, maskA, N)
        elif rounding_strategy=='max_obj':
            v_round = round_by_max_obj_one_threshold(v, z, -1, mask, maskA, N)
        elif rounding_strategy=='bansal':
            v_round = round_by_cc_bansal(z, -1, mask, maskA, N)
        print("HERE after ", v_round[np.where(v_round > 0)])
        print("-------")
        # assign to the new cluster(s)
        for i in range(N):
            if v_round[i]==0: continue
            if z>1:
                if v_round[i]>0: C[i], Y[i,:] = zi, U[zi-1,:].copy() # assign to the zi-th cluster
            else:
                if v_round[i]>0: C[i], Y[i,:] = zi, U[zi-1,:].copy() # assign to the (K-1)-th cluster
                else: C[i], Y[i,:] = zi+1, U[zi,:].copy() # assign to the K-th cluster
        # check current objective value
        print('{}-th iteration obj={:.1f}, x^TAx/x^Tx={:.1f} in ({:.1f}, {:.1f})'.format(
            zi, compute_Obj(Y, A, K), compute_RayleighsQuotient(v_round, maskA), sD, lD))
        # set the assigned nodes to be skipped in the next iteration
        for i in range(N):
            if v_round[i]>0:
                # remove all edges incident to the assigned nodes
                maskA[i,:] = maskA[i,:].multiply(0)
                maskA[:,i] = maskA[:,i].multiply(0)
                mask[i] = 0 # remove the assigned node from the remaining list
    return C, Y, A, N, K

In [57]:
K = 6
C, Y, A, N, K = SCG("bitcoin", K, "min_angle")
check_result_KCG(C, Y, A, N, K, 0)

------ Running bitcoin.txt ------
1-th iteration obj=29.4, x^TAx/x^Tx=30.0 in (-28.1, 46.8)
2-th iteration obj=25.9, x^TAx/x^Tx=13.7 in (-15.8, 23.9)
3-th iteration obj=24.5, x^TAx/x^Tx=14.1 in (-10.4, 17.5)
4-th iteration obj=24.4, x^TAx/x^Tx=8.2 in (-10.4, 14.0)
5-th iteration obj=14.6, x^TAx/x^Tx=7.5 in (-10.3, 13.5)
Obj = 14.6 in (-28.1, 46.8), Execution Time=0.0
|S_1|=143, |In_+|-|In_-|=4412-210, |Out_-|-|Out_+|=960-5615
|S_2|=33, |In_+|-|In_-|=260-0, |Out_-|-|Out_+|=36-4
|S_3|=20, |In_+|-|In_-|=274-0, |Out_-|-|Out_+|=47-159
|S_4|=1, |In_+|-|In_-|=0-0, |Out_-|-|Out_+|=26-2
|S_5|=1, |In_+|-|In_-|=0-0, |Out_-|-|Out_+|=15-12
|S_6|=232, |In_+|-|In_-|=1788-32, |Out_-|-|Out_+|=199-932
|S_0|=5451 // neutral
Total: |S_1|+...+|S_K|=430, |In_+|-|In_-|=6492, |Out_+|-|Out_-|=5441
---------------------------


In [58]:
K = 6
C, Y, A, N, K = SCG2("bitcoin", K, "min_angle")
check_result_KCG(C, Y, A, N, K, 0)

------ Running bitcoin.txt ------
HERE:  [6.12059180e-04 8.59718446e-03 4.56303273e-03 1.83779164e-04
 6.34049016e-05 2.98547765e-03 4.56232687e-05 5.84412305e-04
 1.64615072e-05 9.58262963e-04 1.61576967e-03 5.24929051e-04
 1.24779577e-04 5.03990025e-04 1.12212344e-05 8.16588548e-03
 1.10086959e-04 7.05735222e-06 4.35466667e-06 1.96653840e-04
 4.35466667e-06 2.38112742e-03 3.67792283e-05 4.63894338e-04
 6.00650108e-04 2.99531733e-03 3.25716570e-03 4.16186548e-05
 1.28398984e-05 4.56387789e-05 1.86612601e-06 2.44171514e-04
 4.33513065e-04 1.39841565e-05 1.03113127e-04 5.21957357e-06
 1.15760594e-03 4.39647876e-06 8.55261552e-03 1.82826429e-04
 1.14885317e-04 1.82826429e-04 1.14304125e-03 1.57178732e-04
 1.48668133e-04 4.09306269e-04 1.62477149e-04 6.27449172e-04
 2.64097619e-03 1.42403178e-04 8.65601702e-05 5.61432374e-06
 2.20421516e-06 3.29660143e-03 1.82826429e-04 3.89841654e-04
 1.55381640e-04 3.86680032e-03 3.98915574e-08 2.47457586e-05
 1.21510311e-05 1.82826429e-04 9.75605059e-0